In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [31]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:200])
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

The Wonderful Wizard of Oz

by L. Frank Baum


This book is dedicated to my good friend & comrade
My Wife
L.F.B.


Contents

 Introduction
 Chapter I. The Cyclone
 Chapter II. The Council with the Mun
['\n', ' ', '!', '&', '(', ')', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”']


In [32]:
#Tokenizer

#encoder - transforms an array into an integer
#decoder - transforsm the integer into an array

# If we work with a word level tokenizer then we have a ton of words and that's a lot of tokens
# But in this case we will have a smaller set to encode and decode
# if we have a subwork

string_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_string = { i:ch for i, ch in enumerate(chars)}
encoder = lambda s:[string_to_int[c] for c in s]
decoder = lambda l: ''.join([int_to_string[i] for i in l])

print(encoder('hello'))
print(decoder(encoder('hello')))

[48, 45, 52, 52, 55]
hello


In [33]:
data = torch.tensor(encoder(text), dtype=torch.long)
print(data[:100])

tensor([34, 48, 45,  1, 37, 55, 54, 44, 45, 58, 46, 61, 52,  1, 37, 49, 66, 41,
        58, 44,  1, 55, 46,  1, 29, 66,  0,  0, 42, 65,  1, 26,  8,  1, 20, 58,
        41, 54, 51,  1, 16, 41, 61, 53,  0,  0,  0, 34, 48, 49, 59,  1, 42, 55,
        55, 51,  1, 49, 59,  1, 44, 45, 44, 49, 43, 41, 60, 45, 44,  1, 60, 55,
         1, 53, 65,  1, 47, 55, 55, 44,  1, 46, 58, 49, 45, 54, 44,  1,  3,  1,
        43, 55, 53, 58, 41, 44, 45,  0, 27, 65])


In [59]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) 
    x = torch.stack([data[i:i+block_size] for i in ix]) # This slicing operation extracts a portion of the data defined by the current index and the block size.
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[62, 45, 58,  1, 59, 48, 55, 45],
        [48, 45,  1, 45, 65, 45, 59,  6],
        [54, 44,  0, 52, 55, 55, 51, 45],
        [44,  1, 60, 55,  1, 59, 60, 45]], device='cuda:0')
targets:
tensor([[45, 58,  1, 59, 48, 55, 45, 59],
        [45,  1, 45, 65, 45, 59,  6, 69],
        [44,  0, 52, 55, 55, 51, 45, 44],
        [ 1, 60, 55,  1, 59, 60, 45, 41]], device='cuda:0')


In [35]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [36]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [56]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #This layer is used to map token indices to token embeddings. 
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) #learnable parameter 
   
    # we type it out because we want to know exactly what is going on
    def forward(self, index, targets=None): 
        logits = self.token_embedding_table(index) #It computes the token embeddings for the input index using the self.token_embedding_table.
        # "logits" typically refers to the raw, unnormalized scores or values that are produced by the output layer of a neural network before being passed through a softmax activation function.
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # B - batch / T - time / C channel (vocab size)
            logits = logits.view(B*T, C)  # method is used to change the shape of the tensor while preserving its data
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # c
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C) in this case if shape was 1,1,72 it would take the probs of the 72
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


ljb”zVpQgjFu?—BP
B”QKcnVBS:(1S
!o“’stzX! tJaKXq&g;K—QDU?QN&ZhTcn&A?w9ZWa-IX!’Jund?lvIw1FYJv“kDjhf‘yOAh:?:(rEFouhr‘zqM—CaqOOg&g;fMa?JEK)”Js—w:n‘?QR‘P
Xa1F&PNoWyB’)?xPe“(lnd1mlj(n(ITgFppqadhzhnzyB0UUxn1Frqb‘Nd!,&-nz,xOjEg—:BfxyUfMDl;K)?gYajEwcyIhF,jqX
z-Ok cP(v;S(JnZ—CNwGk-jUxv‘P”au”‘S
AmJTvmg;yB
sdIQzJ1S9yAD”
yvN‘X—xfdxI;?QvtpiT?‘MTeX‘)?QtR&R,&Rdu:Gd;Xh:r.W—‘L.FH
RVTv-Rxu
YwTSou:B.WR:(u;1&xKl,yb0S“FV“rg;QvwtDj‘aPeGK&t,Oh.VhpGi”“kI—itC0UDj
RU bVFW
-W—MRDqfEI“0RUYq1.ixFbgFbgL”zloKpyjOn—Z: U.KkZlf ,


In [63]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        loss = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.458, val loss: 4.472
step: 250, train loss: 4.458, val loss: 4.472
step: 500, train loss: 4.458, val loss: 4.472
step: 750, train loss: 4.458, val loss: 4.472
3.823659658432007
